In [15]:
# !gdown 1Fm6H80FU6p00uVFrrnVzO6khJY6EWq_n # mockup_ecomm_prod_cat-202303.csv
# !gdown 1GRWSd1ZhjCD_zt9eMSDxikmgHSWg3Q2i # mockup_ecomm_vend-202303.csv
# !gdown 1lBGavUDFUQddytiv9vZ5HbAT8wXZut9h # mockup_ecomm_prod-202303.csv
# !gdown 1qhW2CI77AnrB4rlbI1tpJXsE02KHp6l- # mockup_ecomm_cust-202303.csv
# !gdown 1nFSmmgyPBzW5V0sx9NJeJ9MdtqWeDqfX # mockup_ecomm_promo-202303.csv
# !gdown 1klEAZx03UVRqoygTcvucJFYHLma-pEon # mockup_ecomm_prod_promo-202303.csv
# !gdown 19H4Z4wGSpWlfMNBo9VnIkXkmc96ZTL6a # mockup_ecomm_promo_redeem-202303.csv
# !gdown 10ZQ9hBMeL6zsGkzmhegKWzTs-r13uzHN # mockup_ecomm_trx_header-202303.csv
# !gdown 10_GK3FK2PncUAjz2WvLdfwxbdUGZR4N2 # mockup_ecomm_trx_item-202303.csv
# !gdown 1AOV3wY8IJLAfqTzFd0Iy9XNBNKM3U0_S # mockup_ecomm_trx_promo-202303.csv

# !gdown 1JWQ_7u-h-dwGWhdByGF3ARg34XxI8b5h # mockup_ecomm_sales_prod_fact-202303.csv
# !gdown 14Wjn4zoXwYjv-vgVMuTjsjGbtzFN7BZx # mockup_ecomm_sales_promo_fact-202303.csv

Downloading...
From: https://drive.google.com/uc?id=1JWQ_7u-h-dwGWhdByGF3ARg34XxI8b5h
To: /content/mockup_ecomm_sales_prod_fact-202303.csv
100% 13.6M/13.6M [00:00<00:00, 106MB/s] 
Downloading...
From: https://drive.google.com/uc?id=14Wjn4zoXwYjv-vgVMuTjsjGbtzFN7BZx
To: /content/mockup_ecomm_sales_promo_fact-202303.csv
100% 60.9k/60.9k [00:00<00:00, 84.8MB/s]


In [2]:
import pandas as pd;
import numpy as np;
import json;

#Sales-Product-Fact-table

In [3]:
class SaleProdFact:
    pdca:pd.DataFrame = None;
    vend:pd.DataFrame = None;
    pdux:pd.DataFrame = None;

    txit:pd.DataFrame = None;
    txhd:pd.DataFrame = None;

    sale_prod_fact_df:pd.DataFrame = None;

    def set_prodx_cat(self, pdca:pd.DataFrame):
        '''Retrieve columns: `id`, `code`, `label`'''
        self.pdca = pdca[['id', 'code', 'label']].loc[:];
        return self;
    
    def set_vendor(self, vend:pd.DataFrame):
        '''Retrieve columns: `id`, `official_flag`'''
        self.vend = vend[['id', 'official_flag']].loc[:];
        return self;
    
    def set_product(self, pdux:pd.DataFrame):
        '''Retrieve columns: `id`, `vendor_id`, `prod_cat_id`, `name`, `brand`'''
        self.pdux = pdux[['id', 'vendor_id', 'prod_cat_id', 'name', 'brand']].loc[:];
        return self;
    
    def set_trx_header(self, txhd:pd.DataFrame):
        '''Retrieve columns: `id`, `cust_id`, `timestamp`'''
        self.txhd = txhd[['id', 'cust_id', 'timestamp']].loc[:];
        return self;

    def set_trx_item(self, txit:pd.DataFrame):
        '''Retrieve all columns'''
        self.txit = txit.loc[:];
        return self;

    def extract(self) -> pd.DataFrame:
        if self.sale_prod_fact_df is None:
            df:pd.DataFrame = self.txit.set_index('trx_header_id').join(self.txhd.set_index('id'));
            df.reset_index(names=['trx_header_id'], inplace=True);
            df.rename(columns={'id':'trx_item_id', 'quantity': 'trx_item_quantity', 'price':'trx_item_price',
                            'amnt':'trx_item_sales_amnt', 'hrs_incart':'trx_item_hrs_incart', 'timestamp': 'trx_header_timestamp'},
                    inplace=True);

            tmp:pd.DataFrame = self.pdux.set_index('id');
            df.set_index('prod_id', inplace=True);
            df = df.join(tmp);
            df.reset_index(names=['prod_id'], inplace=True);
            df.rename(columns={'name':'prod_name', 'brand':'prod_brand'}, inplace=True);

            tmp = self.pdca.set_index('id');
            df.set_index('prod_cat_id', inplace=True);
            df = df.join(tmp);
            df.reset_index(names=['prod_cat_id'], inplace=True);
            df.rename(columns={'code':'prod_cat_code', 'label':'prod_cat_label'}, inplace=True);

            tmp = self.vend.set_index('id');
            df.set_index('vendor_id', inplace=True);
            df = df.join(tmp);
            df.reset_index(names=['vendor_id'], inplace=True);
            df.rename(columns={'official_flag':'vendor_official_flag'});
            df.sort_values(['trx_item_id'], inplace=True);

            self.sale_prod_fact_df = df[['trx_header_id', 'cust_id', 'trx_item_id', 'prod_id', 'prod_cat_id', 'vendor_id', 'trx_header_timestamp',
                    'prod_name', 'prod_brand', 'prod_cat_code', 'prod_cat_label',
                    'trx_item_quantity', 'trx_item_price', 'trx_item_sales_amnt', 'trx_item_hrs_incart',
                    'official_flag']];
        return self.sale_prod_fact_df.loc[:];

#Sale-Promotion-Fact-table

In [4]:
class SalePromoFact:
    prom: pd.DataFrame = None;
    txhd: pd.DataFrame = None;
    txpr: pd.DataFrame = None;
    redm: pd.DataFrame = None;

    sales_promo_fact_df: pd.DataFrame = None;

    def set_promo(self, prom:pd.DataFrame):
        '''Retrieve columns: `id`, `vendor_id`, `label`'''
        self.prom = prom[['id', 'vendor_id', 'label']].loc[:];
        return self;
    
    def set_promo_redeem(self, redm:pd.DataFrame):
        '''Retrieve columns: `id`, `activated_dt`'''
        self.redm = redm[['id', 'activated_dt']].loc[:];
        return self;

    def set_trx_header(self, txhd:pd.DataFrame):
        '''Retrieve columns: `id`, `cust_id`, `timestamp`'''
        self.txhd = txhd[['id', 'cust_id','timestamp']].loc[:];
        return self;
    
    def set_trx_promo(self, txpr:pd.DataFrame):
        self.txpr = txpr.loc[:];
        return self;

    def extract(self) -> pd.DataFrame:
        if self.sales_promo_fact_df is None:
            df:pd.DataFrame = self.txpr.set_index('trx_header_id').join(self.txhd.set_index('id'));
            df.reset_index(names=['trx_header_id'], inplace=True);
            df.rename(columns={'id':'trx_promo_id', 'discount_amnt':'trx_promo_discount_amnt', 'timestamp': 'trx_header_timestamp'}, inplace=True);

            df.set_index('promo_id', inplace=True);
            df = df.join(self.prom.set_index('id'));
            df.reset_index(names=['promo_id'], inplace=True);
            df.rename(columns={'label':'promo_label'}, inplace=True);

            df.set_index('promo_redeem_id', inplace=True);
            df = df.join(self.redm.set_index('id'));
            df.reset_index(names=['promo_redeem_id'], inplace=True);
            df.rename(columns={'activated_dt':'promo_redeem_activated_dt'}, inplace=True);
            self.sales_promo_fact_df = df[['trx_header_id', 'cust_id', 'promo_id', 'trx_promo_id', 'promo_redeem_id', 'vendor_id',
                    'trx_header_timestamp', 'promo_label', 'trx_promo_discount_amnt', 'promo_redeem_activated_dt']];
            
        return self.sales_promo_fact_df.loc[:];

#Generate Economy Metrix for e-commerce platform

In [5]:
class EconMetrix:
    DAY_RANGE_MAX = pd.Timedelta(360,unit='day');

    LEN_CUT_OFF_INACTIVE = 21 # 0-21 after last purchase
    LEN_CUT_OFF_CHURN = 45 # 21-45 after last purchase

    cust:pd.DataFrame = None;
    vend:pd.DataFrame = None;
    txhd:pd.DataFrame = None;

    sysdate_begin:pd.Timestamp = None;
    sysdate_end:pd.Timestamp = None;

    pref_report:pd.DataFrame = None;

    def set_customer(self, cust:pd.DataFrame):
        df:pd.DataFrame = cust[['id', 'registered_dt']].loc[:];
        df['date'] = pd.to_datetime(df['registered_dt']).dt.date;
        df['date'] = df['date'].apply(pd.Timestamp);
        df.drop(columns=['registered_dt'], inplace=True);
        df.drop_duplicates(inplace=True);
        self.cust = df.loc[:];
        return self;
    
    def set_vendor(self, vend:pd.DataFrame):
        df:pd.DataFrame = vend[['id', 'registered_dt']].loc[:];
        df['date'] = pd.to_datetime(df['registered_dt']).dt.date;
        df['date'] = df['date'].apply(pd.Timestamp);
        df.drop(columns=['registered_dt'], inplace=True);
        self.vend = df.loc[:];
        return self;
    
    def set_trx_header(self, txhd:pd.DataFrame):
        df:pd.DataFrame = txhd.loc[:];
        df['date'] = pd.to_datetime(df['timestamp']).dt.date;
        df['date'] = df['date'].apply(pd.Timestamp);
        self.txhd = df[['cust_id', 'sales_amnt', 'total_discount', 'total_amnt', 'date']].loc[:];
        return self;
    
    def get_txhd(self) -> pd.DataFrame: return self.txhd.loc[:];
    def get_cust(self) -> pd.DataFrame: return self.cust.loc[:];
    def get_vend(self) -> pd.DataFrame: return self.vend.loc[:];


    def init_pref_report(self) -> pd.DataFrame:
        sysdate_begin:pd.Timestamp = self.get_vend()['date'].min();
        sysdate_end:pd.Timestamp = self.get_txhd()['date'].max() + pd.Timedelta(1, unit='day');

        date_range = sysdate_end - sysdate_begin;
        date_range = date_range if date_range < self.DAY_RANGE_MAX else self.DAY_RANGE_MAX;
        sysdate_begin = sysdate_end - date_range;

        self.sysdate_begin = sysdate_begin;
        self.sysdate_end = sysdate_end;

        sys_date:pd.DataFrame = pd.DataFrame([1]*date_range.days, columns=['date']);
        sys_date['delta'] = sys_date.index;
        sys_date['delta'] = sys_date['delta'].apply(lambda x: pd.Timedelta(x, unit='day'));
        sys_date['date'] = self.sysdate_begin + sys_date['delta'];
        sys_date.drop(columns=['delta'], inplace=True);
        return sys_date.loc[:];

    
    def calc_new_vendor(self, pref_report:pd.DataFrame) -> pd.DataFrame:
        # Extract number of new customer
        vend:pd.DataFrame = self.get_vend()
        cnt = vend.groupby('date')[['id']].count();
        cnt.rename(columns={'id':'n_vendor_new'}, inplace=True);
        pref_report = pref_report.set_index('date').join(cnt);
        pref_report.reset_index(names=['date'], inplace=True);
        pref_report['n_vendor_new'] = pref_report['n_vendor_new'].fillna(0);
        return pref_report.loc[:];
    

    def calc_new_customer(self, pref_report:pd.DataFrame) -> pd.DataFrame:
        # Extract number of new customer
        new_cnt = self.get_cust().groupby('date')[['id']].count();
        new_cnt.reset_index(inplace=True);
        new_cnt.rename(columns={'id':'n_cust_new'}, inplace=True);
        new_cnt.set_index('date', inplace=True);
        pref_report.set_index('date', inplace=True);
        pref_report = pref_report.join(new_cnt);
        pref_report.reset_index(names='date', inplace=True);
        pref_report['n_cust_new'] = pref_report['n_cust_new'].fillna(0);
        return pref_report.loc[:];

    
    def calc_customer_stat(self, pref_report:pd.DataFrame) -> pd.DataFrame:
        # Find date range of customer stat
        ## Add referent date - last system date
        rng:pd.DataFrame = self.get_cust();
        tmp = self.get_cust();
        tmp['date'] = self.sysdate_end;
        rng = pd.concat([rng, tmp])

        ## Add active date - assume transaction date
        tmp = self.get_txhd()[['cust_id', 'date']].drop_duplicates();
        tmp.rename(columns={'cust_id':'id'}, inplace=True);
        rng = pd.concat([rng, tmp]);
        rng['date'] = rng['date'].apply(pd.Timestamp)
        rng.drop_duplicates(inplace=True);
        rng.sort_values(by=['id', 'date'], inplace=True);

        ## Finding cutoff date for inactive / churn
        rng['next'] = rng.groupby(['id'])['date'].shift(-1);
        inact_cutoff = pd.Timedelta(self.LEN_CUT_OFF_INACTIVE + 1, unit='day');
        rng['inact_date'] = rng.apply(lambda x: x.date + inact_cutoff
                if (x.date + inact_cutoff < x.next) else pd.NaT, axis='columns');
        churn_cutoff = pd.Timedelta(self.LEN_CUT_OFF_CHURN + 1, unit='day');
        rng['churn_date'] = rng.apply(lambda x: x.date + churn_cutoff
                if (x.date + churn_cutoff < x.next) else pd.NaT, axis='columns');

        ## Add register date - remove new customer
        reg_date = self.get_cust().rename(columns={'date':'reg_date'});
        reg_date['reg_date'] = reg_date['reg_date'].apply(pd.Timestamp);
        reg_date.set_index('id', inplace=True);
        rng.set_index('id', inplace=True);
        rng = rng.join(reg_date);
        rng.reset_index(names=['id'], inplace=True);

        # Extract number of customer by stat
        # Extract number of active customer by active date range
        active = rng[['id', 'reg_date', 'date', 'next', 'inact_date']].loc[:];
        active.rename(columns={'date':'lw_bnd'}, inplace=True);
        active['up_bnd'] = active['inact_date'].combine_first(active.next);
        active.dropna(subset=['up_bnd'], inplace=True)
        active.drop(columns=['next', 'inact_date'], inplace=True);
        active.drop_duplicates(inplace=True);
        pref_report['n_cust_active'] = ( pref_report['date']
                .apply(lambda x: active[ (active.lw_bnd <= x) & (x < active.up_bnd) & (x != active.reg_date)]['id'].drop_duplicates().count()) );

        # Extract number of inactive customer
        inactive = rng[['id', 'date', 'next', 'inact_date', 'churn_date']].dropna(subset=['inact_date'])
        inactive.rename(columns={'inact_date':'lw_bnd'}, inplace=True);
        inactive['up_bnd'] = inactive['churn_date'].combine_first(inactive.next);
        inactive.dropna(subset=['up_bnd'], inplace=True);
        inactive.drop(columns=['next', 'churn_date'], inplace=True);
        inactive.drop_duplicates(inplace=True);
        pref_report['n_cust_inactive'] = ( pref_report['date']
                .apply(lambda x: inactive[ (inactive.lw_bnd <= x) & (x < inactive.up_bnd) ]['id'].drop_duplicates().count()) );
        
        # Extract number of churn customer
        churn = rng[['id', 'date', 'next', 'churn_date']].dropna(subset=['churn_date']);
        churn.rename(columns={'churn_date':'lw_bnd', 'next':'up_bnd'}, inplace=True);
        pref_report['n_cust_churn'] = ( pref_report['date']
                .apply(lambda x: churn[ (churn.lw_bnd <= x) & (x < churn.up_bnd)]['id'].drop_duplicates().count()) );
        
        return pref_report.loc[:];

    
    def calc_trx_amnt(self, pref_report:pd.DataFrame) -> pd.DataFrame:
        txhd = self.get_txhd();

        daily_total = txhd.groupby('date')[['sales_amnt', 'total_discount']].sum();
        daily_total.rename(columns={'sales_amnt':'daily_sales_amnt', 'total_discount':'daily_discount'}, inplace=True);
        pref_report = pref_report.set_index('date').join(daily_total);
        pref_report.fillna(0, inplace=True);

        avg15d = pref_report[['daily_sales_amnt', 'daily_discount']].rolling(15).mean() ## moving avg 15 days
        avg15d = avg15d.round(2);
        avg30d = pref_report[['daily_sales_amnt', 'daily_discount']].rolling(30).mean() ## moving avg 30 days
        avg30d = avg30d.round(2);
        avg45d = pref_report[['daily_sales_amnt', 'daily_discount']].rolling(45).mean() ## moving avg 45 days
        avg45d = avg45d.round(2);

        pref_report = pref_report.join(avg15d, rsuffix='_avg_15d');
        pref_report = pref_report.join(avg30d, rsuffix='_avg_30d');
        pref_report = pref_report.join(avg45d, rsuffix='_avg_45d');
        pref_report.reset_index(names=['date'], inplace=True);
        pref_report.fillna(0, inplace=True);
        
        return pref_report.loc[:];

    
    def calc_est_churn_value(self, pref_report:pd.DataFrame) -> pd.DataFrame:
        # Find customer churn value from median of transaction occured before churn.
        ## Set calculation range begin date and last date of each customer
        cust = self.get_cust();
        tmp = cust.loc[:];
        tmp['date'] = self.sysdate_end;
        cust = pd.concat([cust, tmp]);

        ## calculate median of total amount in each day per customer
        tmp = self.get_txhd()[['cust_id', 'date', 'total_amnt']].loc[:];
        tmp.rename(columns={'cust_id':'id'}, inplace=True);
        cust = pd.concat([cust, tmp]);
        cust.fillna(0, inplace=True);
        cust['date'] = cust['date'].apply(pd.Timestamp);
        cust = cust.groupby(['id', 'date']).median().reset_index(['id', 'date']); #Median
        cust.sort_values(['id', 'date'], inplace=True);
        cust['total_amnt'] = cust['total_amnt'].round(2);

        ## find date range of churn
        cust['next'] = cust.groupby('id')['date'].shift(-1);
        churn_cutoff = pd.Timedelta(self.LEN_CUT_OFF_CHURN + 1, unit='day');
        cust['churn'] = cust.apply(lambda x: x.date + churn_cutoff
                if (x.date + churn_cutoff < x.next) else pd.NaT, axis='columns');
        cust['cutoff_date'] = cust.groupby('id')['churn'].fillna(method='bfill');
        cust['exp_return_date'] = cust['churn'].notna();
        cust['exp_return_date'] = cust.apply(lambda x: x.next if x.exp_return_date else pd.NaT, axis='columns');
        cust['exp_return_date'] = cust.groupby('id')['exp_return_date'].fillna(method='bfill');

        ## Drop zero-values churn customer and date range
        cust.dropna(subset=['cutoff_date'], inplace=True);
        cust = cust.groupby(['id', 'cutoff_date', 'exp_return_date'])[['total_amnt']].median().reset_index(names=['id', 'cutoff_date', 'exp_return_date']);
        cust['total_amnt'] = cust['total_amnt'].round(2);
        cust = cust[cust.total_amnt > 0].loc[:];

        # Sum up churn customer value per day
        pref_report['est_churn_amnt'] = pref_report['date'].apply( lambda x: cust[(cust.cutoff_date <= x) & (x < cust.exp_return_date)].total_amnt.sum() );
        pref_report['est_churn_amnt'] = pref_report['est_churn_amnt'].round(2);

        return pref_report.loc[:];


    def extract(self) -> pd.DataFrame:
        if self.pref_report is None:
            pref_report = self.init_pref_report();
            pref_report = (pref_report.pipe(self.calc_new_vendor)
                    .pipe(self.calc_new_customer)
                    .pipe(self.calc_customer_stat)
                    .pipe(self.calc_est_churn_value)
                    .pipe(self.calc_trx_amnt));
            self.pref_report = pref_report[['date', 'n_vendor_new', 'n_cust_new', 'n_cust_active', 'n_cust_inactive', 'n_cust_churn', 'est_churn_amnt',
                'daily_sales_amnt', 'daily_sales_amnt_avg_15d', 'daily_sales_amnt_avg_30d', 'daily_sales_amnt_avg_45d',
                'daily_discount', 'daily_discount_avg_15d', 'daily_discount_avg_30d', 'daily_discount_avg_45d']];
        return self.pref_report.loc[:];

#Generate Vendor Profile on e-commerce platform

In [6]:
class VendProfile:

    vend:pd.DataFrame = None;
    sale_prodx_fact:pd.DataFrame = None;
    sale_promo_fact:pd.DataFrame = None;

    profile:pd.DataFrame = None;

    def set_vendor(self, df:pd.DataFrame):
        df = df[['id', 'registered_dt']].loc[:];
        df['registered_dt'] = df['registered_dt'].apply(pd.Timestamp);
        df['reg_date'] = df['registered_dt'].dt.date;
        self.vend = df.loc[:];
        return self;

    def set_sale_prodx_fact(self, df:pd.DataFrame):
        df = df.loc[:];
        df['trx_header_timestamp'] = df['trx_header_timestamp'].apply(pd.Timestamp);
        df['date'] = df['trx_header_timestamp'].dt.date;
        df = ( df[['vendor_id', 'date', 'trx_item_sales_amnt']]
                .groupby(['vendor_id', 'date'])
                .sum()
                .reset_index() );
        self.sale_prodx_fact = df.rename(columns={'vendor_id':'id', 'trx_item_sales_amnt':'sales_amnt'}).loc[:];
        return self;
    
    def set_sale_promo_fact(self, df:pd.DataFrame):
        df = df.loc[:];
        df['trx_header_timestamp'] = df['trx_header_timestamp'].apply(pd.Timestamp);
        df['date'] = df['trx_header_timestamp'].dt.date;
        df = ( df[['vendor_id', 'date', 'trx_promo_discount_amnt']]
                .groupby(['vendor_id', 'date'])
                .sum()
                .reset_index() )
        self.sale_promo_fact = df.rename(columns={'vendor_id':'id', 'trx_promo_discount_amnt':'discount_amnt'}).loc[:];
        return self;

    def get_vendor(self) -> pd.DataFrame: return self.vend.loc[:];
    def get_sale_prodx_fact(self) -> pd.DataFrame: return self.sale_prodx_fact.loc[:];
    def get_sale_promo_fact(self) -> pd.DataFrame: return self.sale_promo_fact.loc[:];

    def init_ref_sysdates(self) -> pd.DataFrame:
        limit_range = pd.Timedelta(45,unit='days');

        # ref_vend:pd.DataFrame = self.get_vendor()['reg_date'];
        ref_prodx_fact:pd.DataFrame = self.get_sale_prodx_fact()['date'];
        ref_promo_fact:pd.DataFrame = self.get_sale_promo_fact()['date'];

        # end_refdate = pd.concat([ref_vend, ref_prodx_fact, ref_promo_fact]).max()
        end_refdate = pd.concat([ref_prodx_fact, ref_promo_fact]).max()
        start_refdate = pd.concat([ref_prodx_fact, ref_promo_fact]).min();
        start_refdate = ( start_refdate if (end_refdate - start_refdate).days <= 44 else
                end_refdate - pd.Timedelta(44, unit='day') );
        
        return pd.DataFrame([start_refdate + pd.Timedelta(i, 'day') for i in range(45)], columns=['date']);

    def extract(self) -> pd.DataFrame:
        if self.profile is None:
            ref_sysdates = self.init_ref_sysdates();
            vendors = self.get_vendor();
            
            df:pd.DataFrame = vendors[['id']].join(ref_sysdates, how='cross');
            df = ( df.set_index(['id', 'date'])
                .join(self.get_sale_prodx_fact().set_index(['id', 'date']))
                .join(self.get_sale_promo_fact().set_index(['id', 'date']))
                .reset_index(names=['id', 'date']) );
            df.fillna(0, inplace=True);
            
            df['sales_mvavg05'] = df.groupby(['id'])['sales_amnt'].transform(lambda x: x.rolling(5,1).mean());
            df['sales_mvavg05'] = df['sales_mvavg05'].round(2);
            df['sales_mvavg10'] = df.groupby(['id'])['sales_amnt'].transform(lambda x: x.rolling(10,1).mean());
            df['sales_mvavg10'] = df['sales_mvavg10'].round(2);

            df['dc_mvavg05'] = df.groupby(['id'])['discount_amnt'].transform(lambda x: x.rolling(5,1).mean());
            df['dc_mvavg05'] = df['dc_mvavg05'].round(2);
            df['dc_mvavg10'] = df.groupby(['id'])['discount_amnt'].transform(lambda x: x.rolling(10,1).mean());
            df['dc_mvavg10'] = df['dc_mvavg10'].round(2);
            
            self.profile = df;
        return self.profile.loc[:];

#Generate Customer Profile on e-commerce platform

In [7]:
class CustProfile:
    LIMIT_RANGE_DAYS = pd.Timedelta(89,unit='day'); #90 days, included last date
    LEN_CUT_OFF_INACTIVE = 21 # 0-14 after last purchase
    LEN_CUT_OFF_CHURN = 45 # 15-21 after last purchase

    RECENT_FLAG_ACTIVE:str = 'active';
    RECENT_FLAG_INACTIVE:str = 'inactive';
    RECENT_FLAG_CHURN:str = 'churn';
    RECENT_FLAG_UNKNOWN:str = 'unknown';

    cust:pd.DataFrame = None;
    sales_prodx_fact:pd.DataFrame = None;
    sales_promo_fact:pd.DataFrame = None;

    profile: pd.DataFrame = None;
    
    def set_customer(self, df:pd.DataFrame):
        df = df[['id', 'registered_dt']].drop_duplicates(subset=['id']);
        df['registered_dt'] = df['registered_dt'].apply(pd.Timestamp);
        df['reg_date'] = df['registered_dt'].dt.date;
        df.drop(columns=['registered_dt'], inplace=True);
        self.cust = df.loc[:];
        return self;
    
    def set_sale_prodx_fact(self, df:pd.DataFrame):
        df = df[['cust_id', 'prod_cat_id', 'trx_item_sales_amnt', 'trx_header_timestamp']].loc[:];
        df['trx_header_timestamp'] = df['trx_header_timestamp'].apply(pd.Timestamp);
        df['date'] = df['trx_header_timestamp'].dt.date;
        df = ( df.groupby(['cust_id', 'date', 'prod_cat_id'])['trx_item_sales_amnt'].sum()
                .reset_index() );
        df['trx_item_sales_amnt'] = df['trx_item_sales_amnt'].round(2);
        df.rename(columns={'cust_id':'id', 'trx_item_sales_amnt':'sales_amnt'}, inplace=True);
        
        self.sales_prodx_fact = df.loc[:];
        return self;
    
    def set_sale_promo_fact(self, df:pd.DataFrame):
        df = df[['cust_id', 'trx_header_timestamp', 'trx_promo_discount_amnt']].loc[:];
        df['trx_header_timestamp'] = df['trx_header_timestamp'].apply(pd.Timestamp);
        df['date'] = df['trx_header_timestamp'].dt.date;
        df = ( df.groupby(['cust_id', 'date'])['trx_promo_discount_amnt'].sum()
                .reset_index() );
        df['trx_promo_discount_amnt'] = df['trx_promo_discount_amnt'].round(2);
        df.rename(columns={'cust_id':'id', 'trx_promo_discount_amnt':'dc_amnt'}, inplace=True);

        self.sales_promo_fact = df.loc[:];
        return self;
    
    def get_customer(self) -> pd.DataFrame: return self.cust.loc[:];
    def get_sale_prodx_fact(self) -> pd.DataFrame: return self.sales_prodx_fact.loc[:];
    def get_sale_promo_fact(self) -> pd.DataFrame: return self.sales_promo_fact.loc[:];

    def extract(self) -> pd.DataFrame:
        def flag_recent_stat(day_diff:int, patron_length:int) -> str:
            if (self.LEN_CUT_OFF_CHURN < day_diff): return self.RECENT_FLAG_CHURN;
            if self.LEN_CUT_OFF_INACTIVE < day_diff <= self.LEN_CUT_OFF_CHURN: return self.RECENT_FLAG_INACTIVE
            if day_diff <= self.LEN_CUT_OFF_INACTIVE: return self.RECENT_FLAG_ACTIVE;
            return self.RECENT_FLAG_UNKNOWN;

        def cal_purchase_prob(n_purchase:int, patron_length:int) -> float:
            if patron_length == 0: return np.nan;
            length = patron_length if patron_length <= self.LIMIT_RANGE_DAYS.days else 90;
            return n_purchase/length;
            
        def cal_category_prob(cat_ls:list, n_purchase:int) -> dict:
            df:pd.DataFrame = pd.DataFrame({
                'cat_id': cat_ls.split(','),
                'category_prob': 1
            });
            df = df.groupby('cat_id')['category_prob'].sum().reset_index();
            df['category_prob'] = (df['category_prob']/n_purchase).round(2);
            if len(df.index) < 6: return df.set_index('cat_id').to_json();
            q3 = df['category_prob'].quantile(.75);
            q4 = df['category_prob'].quantile(1)
            return df[ df['category_prob'].between(q3, q4) ].set_index('cat_id').to_json();

        
        if self.profile is None:
            # Retrieve sales amount - discount per day
            sales:pd.DataFrame = self.get_sale_prodx_fact();
            discount:pd.DataFrame = self.get_sale_promo_fact();

            # initial data range
            sysdate_start = pd.concat([ sales['date'], discount['date'] ]).min();
            sysdate_end = pd.concat([ sales['date'], discount['date'] ]).max();
            sysdate_start = sysdate_end - self.LIMIT_RANGE_DAYS if (sysdate_end - sysdate_start).days > 89 else sysdate_start;
            
            # Retrieve data in range
            ## Retrieve customer
            customer = self.get_customer();
            customer['patron_length'] = (sysdate_end - customer['reg_date']).dt.days;

            ## Retrieve sales info
            sales = sales[ sales['date'].between(sysdate_start, sysdate_end) ].loc[:];

            ### Customer amount on purchased day - pocket size
            purchased_amnt:pd.DataFrame = sales.groupby(['id','date'])['sales_amnt'].sum().reset_index();
            purchased_amnt['sales_amnt'] = purchased_amnt['sales_amnt'].round(2);
            sales_median = sales.groupby(['id'])['sales_amnt'].median().reset_index();
            sales_median['sales_amnt'] = sales_median['sales_amnt'].round(2);
            sales_median.rename(columns={'sales_amnt':'pkt_sz'}, inplace=True);
            customer = customer.set_index('id').join(sales_median.set_index('id')).reset_index();
            customer['pkt_sz'] = customer['pkt_sz'].fillna(0);

            ### Calculate gap btw purchase / recency / probability of purchase & product category
            purchased_date:pd.DataFrame = purchased_amnt[['id','date']].loc[:];
            purchased_date['n_purchase'] = 1;
            purchased_date['next'] = purchased_date.groupby('id')['date'].shift(-1);
            purchased_date['gap'] = (purchased_date['next'] - purchased_date['date']).dt.days;

            #### Gap btw purchase
            gap_btw_purchase = purchased_date.groupby('id')['gap'].median().reset_index();
            customer = customer.set_index('id').join(gap_btw_purchase.set_index('id')).reset_index();
            customer['gap'] = customer['gap'].fillna(0)

            #### Recency Flag
            latest_purchase = purchased_date.groupby('id')['date'].max().reset_index();
            latest_purchase.rename(columns={'date':'latest_purchase'}, inplace=True);
            customer = customer.set_index('id').join(latest_purchase.set_index('id')).reset_index();
            customer['refdate_diff'] = (sysdate_end - customer['latest_purchase']).dt.days;
            customer['recent_flag'] = customer[['patron_length', 'refdate_diff']].apply( lambda x:
                    flag_recent_stat(x['refdate_diff'], x['patron_length']), axis='columns' );
            customer.drop(columns=['refdate_diff'], inplace=True);

            #### Probability of purchase
            n_purchase_in_period = purchased_date.groupby('id')['n_purchase'].sum().reset_index();
            customer = customer.set_index('id').join(n_purchase_in_period.set_index('id')).reset_index();
            customer['n_purchase'] = customer['n_purchase'].fillna(0);
            customer['p_purchase'] = customer[['n_purchase', 'patron_length']].apply( lambda x: 
                    cal_purchase_prob(x['n_purchase'], x['patron_length']), axis='columns' );
            customer['p_purchase'] = customer['p_purchase'].round(2);
            customer['p_purchase'] = customer['p_purchase'].fillna(0);
            customer.drop(columns=['n_purchase'], inplace=True);

            #### Customer purchased category probability
            purchased_cat:pd.DataFrame = sales.groupby(['id'])['prod_cat_id'].apply(','.join).reset_index();
            purchased_cat = ( purchased_cat.set_index('id').join(n_purchase_in_period.set_index('id'))
                    .reset_index() );
            purchased_cat['category_p'] = purchased_cat[['prod_cat_id', 'n_purchase']].apply(lambda x:
                    cal_category_prob(x['prod_cat_id'], x['n_purchase']), axis='columns');
            purchased_cat.drop(columns=['prod_cat_id', 'n_purchase'], inplace=True);
            customer = customer.set_index('id').join(purchased_cat.set_index('id')).reset_index();

            ## Retrieve redeem discount
            discount = discount[ discount['date'].between(sysdate_start, sysdate_end) ].loc[:];
            discount['#'] = 1;

            ### Redeemed purchase probability
            p_dc:pd.DataFrame = discount.groupby('id')['#'].sum().reset_index();
            p_dc = p_dc.set_index('id').join( n_purchase_in_period.set_index('id') ).reset_index();
            p_dc['p_purchase_dc'] = p_dc['#']/p_dc['n_purchase'];
            p_dc['p_purchase_dc'] = p_dc['p_purchase_dc'].round(2);
            customer = ( customer.set_index('id').join(p_dc[['id', 'p_purchase_dc']].set_index('id'))
                    .reset_index(names=['id']) );
            customer['p_purchase_dc'] = customer['p_purchase_dc'].fillna(0);

            ### Discounted amount
            dc_median:pd.DataFrame = discount.groupby('id')['dc_amnt'].median().reset_index();
            dc_median['dc_amnt'] = dc_median['dc_amnt'].round(2);
            customer = ( customer.set_index('id').join(dc_median.set_index('id'))
                    .reset_index(names=['id']) );
            customer['dc_amnt'] = customer['dc_amnt'].fillna(0);

            self.profile = customer[ ['id', 'reg_date', 'patron_length', 'latest_purchase', 'recent_flag',
                    'pkt_sz', 'dc_amnt', 'gap', 'p_purchase', 'p_purchase_dc', 'category_p'] ];
        return self.profile.loc[:];

# profile = ( CustProfile().set_customer(pd.read_csv('mockup_ecomm_cust-202303.csv'))
#     .set_sale_prodx_fact(pd.read_csv('mockup_ecomm_sales_prod_fact-202303.csv'))
#     .set_sale_promo_fact(pd.read_csv('mockup_ecomm_sales_promo_fact-202303.csv')) );
# customer = profile.extract();
# customer.to_csv('CUST_PROFILE.csv', index=False)
# customer
# customer[ customer['category_p'].notna() ].set_index('id')['category_p'].apply(lambda x: json.loads(x)['category_prob']).transform(pd.Series) 

#Test

In [8]:
# (SaleProdFact()
#     .set_prodx_cat(pd.read_csv('mockup_ecomm_prod_cat-202303.csv'))
#     .set_vendor(pd.read_csv('mockup_ecomm_vend-202303.csv'))
#     .set_product(pd.read_csv('mockup_ecomm_prod-202303.csv'))
#     .set_trx_header(pd.read_csv('mockup_ecomm_trx_header-202303.csv'))
#     .set_trx_item(pd.read_csv('mockup_ecomm_trx_item-202303.csv'))
#     .extract())

In [9]:
# (SalePromoFact()
#     .set_promo(pd.read_csv('mockup_ecomm_promo-202303.csv'))
#     .set_promo_redeem(pd.read_csv('mockup_ecomm_promo_redeem-202303.csv'))
#     .set_trx_header(pd.read_csv('mockup_ecomm_trx_header-202303.csv'))
#     .set_trx_promo(pd.read_csv('mockup_ecomm_trx_promo-202303.csv'))
#     .extract())

In [14]:
# (EconMetrix()
#     .set_vendor(pd.read_csv('mockup_ecomm_vend-202303.csv'))
#     .set_customer(pd.read_csv('mockup_ecomm_cust-202303.csv'))
#     .set_trx_header(pd.read_csv('mockup_ecomm_trx_header-202303.csv'))
#     .extract())

In [16]:
# (VendProfile()
#     .set_vendor(pd.read_csv('mockup_ecomm_vend-202303.csv'))
#     .set_sale_prodx_fact(pd.read_csv('mockup_ecomm_sales_prod_fact-202303.csv'))
#     .set_sale_promo_fact(pd.read_csv('mockup_ecomm_sales_promo_fact-202303.csv'))
#     .extract())

In [12]:
# (CustProfile().set_customer(pd.read_csv('mockup_ecomm_cust-202303.csv'))
#     .set_sale_prodx_fact(pd.read_csv('mockup_ecomm_sales_prod_fact-202303.csv'))
#     .set_sale_promo_fact(pd.read_csv('mockup_ecomm_sales_promo_fact-202303.csv'))
#     .extract())